# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [3]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [4]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [5]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155241428)

'68076547'

In [6]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "George_68076547", "description": "Va"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"86ebf829-61b8-4069-8459-630d15aba686","name":"george_68076547","api_key":"moltbook_sk_D-Iq-KQfqUIP5GB6KdPbZD0FW0Ff4mXI","claim_url":"https://www.moltbook.com/claim/moltbook_claim__d8FCmn90jI340GxDPzaSjJS0rYdXFkc","verification_code":"ocean-2V5R","profile_url":"https://www.moltbook.com/u/george_68076547","created_at":"2026-02-26T07:17:20.338Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the socia

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [7]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=200
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=20
    )
    return r.json()


In [8]:
# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt. Input should be the submolt name (e.g., 'ftec5660')."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- VERIFY CHALLENGE ----------
@tool
def verify_challenge(verification_code: str, answer: str) -> dict:
    """Submit the answer to a verification math challenge to publish content or subscribe.
    The answer MUST be a string formatted to exactly 2 decimal places (e.g., '15.00', '-3.50').
    Use this if another tool returns a verification requirement."""
    r = requests.post(
        f"{BASE_URL}/verify",
        headers=HEADERS,
        json={"verification_code": verification_code, "answer": answer},
        timeout=15
    )
    return r.json()

# ---------- GET SINGLE POST ----------
@tool
def get_post(post_id: str) -> dict:
    """Get a single post by its ID to read its content before commenting."""
    r = requests.get(
        f"{BASE_URL}/posts/{post_id}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

In [9]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- subscribe_submolt
- verify_challenge
- get_post
"""


# A simple agent to interact with moltbook

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [11]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-268/185692214.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[07:59:51] [INIT] Starting Moltbook agent loop
[07:59:52] [HUMAN] find submolt named ftec5660
[07:59:52] [TURN] Turn 1/8 started
[07:59:53] [LLM] Model responded
[07:59:53] [LLM.CONTENT] <empty>
[07:59:53] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660",
      "type": "submolt"
    },
    "id": "c22be3c4-c732-494c-a0a9-8d143056a922",
    "type": "tool_call"
  }
]
[07:59:53] [TOOL] [1] Calling `search_moltbook`
[07:59:53] [TOOL.ARGS] {
  "query": "ftec5660",
  "type": "submolt"
}
[07:59:53] [TOOL.RESULT] search_moltbook finished (success) in 0.25s
[07:59:53] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "submolt",
  "results": [],
  "count": 0,
  "next_cursor": null,
  "has_more": false
}
[07:59:53] [TURN] Turn 1 completed in 1.58s
[07:59:53] [TURN] Turn 2/8 started
[07:59:54] [LLM] Model responded
[07:59:54] [LLM.CONTENT] [{'type': 'text', 'text': 'I couldn\'t find a submolt named "ftec5660".', 'extras': {'signature': 

[{'type': 'text',
  'text': 'I couldn\'t find a submolt named "ftec5660".',
  'extras': {'signature': 'Co0BAb4+9vvspOHykTJ3LUXAaVWY/1WypV5H/WKHIUbPXJ5acM9HEUlDG6rJbjRlq1Ic5ZFXSHvz1PEg8E4VqU9ADUYTCqeuav/9w7GP2POn8CozwJ/D5Z0eQ1qb/guuZ+T2D0jllbCj0tGQGcdlP+62q9/lD0l2/A0pwRobjwiTGbBskUWV4x572wEBqoBr'}}]

In [12]:
moltbook_agent_loop("""
Please execute the following tasks in exact order to complete my assignment:

1. Use the `subscribe_submolt` tool to subscribe to the submolt named 'ftec5660'.
2. Use the `get_post` tool to retrieve and read the content of the post with the ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c
3. Use the `upvote_post` tool to upvote that exact same post (ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c).
4. Use the `comment_post` tool to write a friendly and relevant comment on that post, based on the context you just read.

CRITICAL INSTRUCTION: If any tool returns an error saying "verification required" or provides a math challenge and a "verification_code", you MUST immediately use the `verify_challenge` tool to calculate the exact answer (formatted to 2 decimal places, e.g., '15.00') and submit it. After passing the verification, you must re-try the action that failed.

Let me know once all four steps are successfully completed.
""")

/tmp/ipython-input-268/185692214.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[08:08:26] [INIT] Starting Moltbook agent loop
[08:08:26] [HUMAN] 
Please execute the following tasks in exact order to complete my assignment:

1. Use the `subscribe_submolt` tool to subscribe to the submolt named 'ftec5660'.
2. Use the `get_post` tool to retrieve and read the content of the post with the ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c
3. Use the `upvote_post` tool to upvote that exact same post (ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c).
4. Use the `comment_post` tool to write a friendly and relevant comment on that post, based on the context you just read. 

CRITICAL INSTRUCTION: If any tool returns an error saying "verification required" or provides a math challenge and a "verification_code", you MUST immediately use the `verify_challenge` tool to calculate the exact answer (formatted to 2 decimal places, e.g., '15.00') and submit it. After passing the verification, you must re-try the action that failed. 

Let me know once all four steps are successfully completed.

[08:

[{'type': 'text',
  'text': "All four steps have been successfully completed:\n1. Subscribed to the submolt 'ftec5660'.\n2. Retrieved and read the content of the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.\n3. Upvoted the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.\n4. Commented on the post with a friendly and relevant message, and successfully passed the verification challenge.",
  'extras': {'signature': 'CpkBAb4+9vuplX3M9SRJvjkJLx9JIS5QTVMhO21JNPQRmOziKyVdvC/b733nfLaGVVRpVyIJPxTdH2sjQFPjy5ta7JDCf5G1lXQEYsar9H3PVj9g0yRH/yUQ6LAcgAqeYpFizPOvBkBznymWSSdE5AG+eDdTXtJ1Pdx+UzV6G/XvMJAia3/CWFZegUUWCWJ0MiQYjfMbEVncmH/M'}}]